In [2]:

# =============================================================================
# Agricultural and Income Questionnaire: Checks and Balances
# =============================================================================
import numpy as np
import pandas as pd
import os 
os.chdir('C:/Users/rodri/Dropbox/Chied_Field_June_19/Data/Income')

# =============================================================================
# Import data
# =============================================================================
data_test = pd.read_stata("Agriculture testing-dataset.dta", convert_categoricals=False)
data_test.rename(columns={'householdid':'hhid'}, inplace=True) 
data_test.rename(columns={'interviewername':'enumerator'}, inplace=True) 
#data20 = pd.read_stata("income_200719.dta", convert_categoricals=False)
# Last version already includes previous interviews. Except for the testing.
data21 = pd.read_stata("income_230719.dta", convert_categoricals=False)

datasets = [data_test, data21]

data = pd.concat(datasets, sort=False)
data.reset_index(drop=True, inplace=True)

percentiles = [0.05, 0.1, .25, .5, .75, 0.8, 0.9, 0.95, 0.99]

list_crops = ['maize', 'groundnut', 'groundbean', 'sweetpotatoe', 'fingermillet', 'sorghum', 'pearlmillet', 'soyabean', 'pigeonpeas', 'cotton', 'nkhwani', 'cassava',  'sugarcane',  'tomatoes',  'therereokra', 'tanaposi' ] 

# Rename some variables
data.rename(columns={'unitssoldpearlmillet2':'unitssoldpearlmilletout2'}, inplace=True) 
data.rename(columns={'unitssoldsoyabean2':'unitssoldsoyabeanout2'}, inplace=True)
data.rename(columns={'soldquantitygroundbeanin':'soldquantitygroundbeanin'}, inplace=True)

## Remove 9999 observations=====================================
data.replace([9999, 9999.00], np.nan, inplace=True)




# =============================================================================
# Check number, size, and value plots
# =============================================================================
data['total_plots'] = data['manyplot'].fillna(0) + data['rentinmany'].fillna(0)
sumplots = data[['manyplot','rentinmany','total_plots']].describe(percentiles=percentiles)
N_plots = int(data[['total_plots']].max())

print('===========================================')
print('Summary number of plots')
sumplots
##### STOP RUN    

Summary number of plots


,manyplot,rentinmany,total_plots
count,115.000000,10.0,115.000000
mean,1.800000,1.0,1.886957
std,1.069514,0.0,1.049354
min,0.000000,1.0,0.000000
5%,1.000000,1.0,1.000000
10%,1.000000,1.0,1.000000
25%,1.000000,1.0,1.000000
50%,2.000000,1.0,2.000000
75%,2.000000,1.0,2.000000
80%,2.000000,1.0,2.000000


In [3]:

## Check plot size and value =========================================
#units area plots
units_plot = pd.value_counts(data['unitsareaplot_1'])
# small futbol fields are around 1 acre
# square meters to acres: 0.000247105

for i in range(1,N_plots+1):
    data['area_plot_acr_'+str(i)] = data['areaplot_'+str(i)]
    data.loc[data['unitsareaplot_'+str(i)]==2.0, 'area_plot_acr_'+str(i)] = data.loc[data['unitsareaplot_'+str(i)]==2.0, 'areaplot_'+str(i)]*2.47105
    data.loc[data['unitsareaplot_'+str(i)]==4.0, 'area_plot_acr_'+str(i)] = data.loc[data['unitsareaplot_'+str(i)]==4.0, 'areaplot_'+str(i)]*0.000247105

for i in range(1,N_plots+1):
    data['ratio_value_rent_'+str(i)] = np.nan
    data['p_acre_plot_'+str(i)]= np.nan

#Check ratio value vs rentout: 
for i in range(1,N_plots+1):
    data['ratio_value_rent_'+str(i)]  =  data['valueplot_'+str(i)] / data['rentoutplot_'+str(i)] 

## Check price per acre:
for i in range(1,N_plots+1):
    data['p_acre_plot_'+str(i)]  =  data['valueplot_'+str(i)] / data['area_plot_acr_'+str(i)] 
    
# hh aggregate variables
data['hh_area_plots'] = 0
data['hh_rentout_plots'] = 0
data['hh_value_plots'] = 0

    
### Add at household level:    
for i in range(1,N_plots+1):
    data['hh_area_plots'] += data['area_plot_acr_'+str(i)].fillna(0)
    data['hh_rentout_plots'] += data['rentoutplot_'+str(i)].fillna(0)
    data['hh_value_plots'] += data['valueplot_'+str(i)].fillna(0)


data['hh_p_acre_plots']  =  data['hh_value_plots'] / data['hh_area_plots'] 
data['hh_ratio_value_rent'] = data['hh_value_plots'] / data['hh_rentout_plots'] 


    
print('===============================================================')
print('Check: Distribution First Reported Plot')
print('===============================================================')    
sum_1plot = data[['area_plot_acr_1','rentoutplot_1','valueplot_1', 'ratio_value_rent_1', 'p_acre_plot_1']].describe(percentiles=percentiles)
sum_1plot
### STOP RUN

Check: Distribution First Reported Plot


,area_plot_acr_1,rentoutplot_1,valueplot_1,ratio_value_rent_1,p_acre_plot_1
count,114.000000,104.000000,9.500000e+01,93.000000,9.500000e+01
mean,2.970073,16793.461538,2.953789e+05,19.028299,1.638863e+05
std,9.502050,16885.719236,5.645670e+05,33.207146,3.128093e+05
min,0.000000,0.000000,9.990000e+02,1.000000,6.660000e+02
5%,0.500000,2875.000000,3.000000e+04,3.200000,1.000000e+04
10%,0.965000,5000.000000,5.000000e+04,4.328571,3.094996e+04
25%,1.000000,8000.000000,1.000000e+05,7.142857,7.321429e+04
50%,1.650000,10000.000000,1.500000e+05,12.500000,1.000000e+05
75%,2.500000,20000.000000,3.000000e+05,20.000000,1.666667e+05
80%,2.500000,20000.000000,3.000000e+05,20.000000,2.000000e+05


In [4]:

print('===============================================================')
print('Check: Distribution Second Reported Plot')
print('===============================================================')    
sum_2plot = data[['area_plot_acr_2','rentoutplot_2','valueplot_2', 'ratio_value_rent_2','p_acre_plot_2']].describe(percentiles=percentiles)
sum_2plot
##### STOP RUN




Check: Distribution Second Reported Plot


,area_plot_acr_2,rentoutplot_2,valueplot_2,ratio_value_rent_2,p_acre_plot_2
count,62.000000,54.000000,5.000000e+01,49.000000,5.000000e+01
mean,1.212106,10685.833333,1.686000e+05,134.666667,2.683145e+05
std,0.747224,10023.934929,2.525423e+05,814.770398,6.598863e+05
min,0.002471,0.000000,5.000000e+03,2.000000,1.000000e+04
5%,0.256500,1312.250000,2.000000e+04,4.000000,2.010544e+04
10%,0.410000,3000.000000,2.000000e+04,4.000000,3.960000e+04
25%,0.525000,5000.000000,5.000000e+04,5.000000,6.666667e+04
50%,1.000000,8000.000000,1.000000e+05,12.000000,1.000000e+05
75%,1.500000,14750.000000,1.800000e+05,18.571429,1.618750e+05
80%,1.500000,17000.000000,2.000000e+05,20.000000,2.000000e+05


In [5]:

# =============================================================================
# Check: land area, rentout value, and land value at household level
# =============================================================================
sum_hhplots = data[['hh_area_plots','hh_rentout_plots','hh_value_plots', 'hh_ratio_value_rent', 'hh_p_acre_plots']].describe(percentiles=percentiles)
print('')
print('===============================================================')
print('Check: Distribution land at household level')
print('===============================================================')
sum_hhplots

### STOP RUN


Check: Distribution land at household level


,hh_area_plots,hh_rentout_plots,hh_value_plots,hh_ratio_value_rent,hh_p_acre_plots
count,115.000000,115.000000,1.150000e+02,102.000000,1.130000e+02
mean,3.956971,22335.260870,3.447043e+05,inf,1.365886e+05
std,9.707566,26695.742752,6.383736e+05,NaN,3.033692e+05
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
5%,0.675000,0.000000,0.000000e+00,0.050000,0.000000e+00
10%,1.000000,0.000000,0.000000e+00,3.343750,0.000000e+00
25%,1.233649,8750.000000,7.000000e+04,6.666667,3.237490e+04
50%,2.471050,15000.000000,1.900000e+05,11.715686,8.333333e+04
75%,3.500000,25000.000000,3.600000e+05,19.145299,1.428571e+05
80%,4.500000,30000.000000,4.300000e+05,20.000000,1.666667e+05


In [6]:

#CHeck    pd.value_counts(data['area_plot_acr_1'])
#Check    pd.value_counts(data['area_plot_acr_2'])

## REINTERVIEW THOSE HOUSEHOLDS WITH EXTREME VALUES:
big_areas = data.loc[data['hh_area_plots']>5,['enumerator','intervieweename','hh_area_plots','hh_rentout_plots','hh_value_plots']]
print('')
print('===============================================================')
print('Check: Potential outliers land size')
print('===============================================================')
big_areas
## add value per acre. Add value/rentout. (later add yields: kgs per acre)
### STOP RUN



Check: Potential outliers land size


,enumerator,intervieweename,hh_area_plots,hh_rentout_plots,hh_value_plots
20,2,Lakia James,6.206575,17000.0,150000.0
23,6,Esnart James,25.000000,30000.0,100000.0
30,5,Ainess Samson,5.000000,122000.0,680000.0
34,5,Esme Kalimu,5.200000,34000.0,500000.0
41,4,Lukiya Alaki,15.000000,75000.0,1070000.0
45,2,Agness Amos,5.600000,95000.0,420000.0
56,7,Zione kalipinde,7.700000,82035.0,750000.0
60,6,Lena Julius,101.250000,40000.0,350000.0
73,3,Mina Rashid,7.600000,61000.0,1000000.0
75,7,Magret Kumwima,12.300000,8000.0,1650000.0


In [7]:

# =============================================================================
#  Summarize land rights 
# =============================================================================

sum_landrights = data[['rightsellland', 'rightbequeathplot', 'chiefpreventsell', 'chiefpreventbequeat', 'landdispute' ]].describe()
sum_landrights
### STOP RUN


,rightsellland,rightbequeathplot,chiefpreventsell,chiefpreventbequeat,landdispute
count,113.000000,113.000000,113.000000,113.000000,113.000000
mean,1.557522,1.557522,1.858407,1.867257,1.849558
std,0.498893,0.498893,0.350185,0.340809,0.359097
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,2.000000,2.000000,2.000000
50%,2.000000,2.000000,2.000000,2.000000,2.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000
max,2.000000,2.000000,2.000000,2.000000,2.000000


In [8]:
#%% Convert agricultural outputs to kgs
# =============================================================================

# Import conversion rates
crop_unit = pd.read_csv("crop_conversions_kg.csv")
crop_unit.set_index('unit', inplace=True)

#Generate empty variables
for crop in list_crops:
    data['total_kg_'+crop] = np.nan 
    data['sold_kg_'+crop] = np.nan 
    data['sold_insiders_kg_'+crop] = np.nan 
    data['sold_outsiders_kg_'+crop] = np.nan 
    data['sold_outside_kg_'+crop] = np.nan 
    data['store_kg_'+crop] = np.nan 
    data['lost_kg_'+crop] = np.nan
    data['total2_kg_'+crop] =np.nan
    data['sold_bigger_total_'+crop] = 0
    data['lost_bigger_total_'+crop] = 0
    data['store_bigger_total_'+crop] = 0
    data['soldloststore_bigger_total_'+crop] = 0
    data['p_'+crop] = np.nan
    data['y_'+crop] = 0
    data['y_agric'] = 0
    data[['unitstotal'+crop, 'unitssold'+crop, 'unitsstore'+crop,'unitslost'+crop]].replace(np.nan, 0, inplace=True)

    
       
# =============================================================================
# Main Loop: Conversion to kgs for all crops and questions    

data.replace(np.nan, 0, inplace=True)
for i in range(len(data)): 
    for crop in list_crops:
        data.iloc[i, data.columns.get_loc('total_kg_'+crop)] = data.iloc[i,data.columns.get_loc('totalamount'+crop)]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitstotal'+crop)]),'conversionkg']
        data.iloc[i, data.columns.get_loc('sold_kg_'+crop)] = data.iloc[i,data.columns.get_loc('soldquantity'+crop)]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitssold'+crop)]),'conversionkg']
        data.iloc[i, data.columns.get_loc('sold_insiders_kg_'+crop)] = data.iloc[i,data.columns.get_loc('soldquantity'+crop+'in')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitssold'+crop+'in')]),'conversionkg']
        data.iloc[i, data.columns.get_loc('sold_outsiders_kg_'+crop)] = data.iloc[i,data.columns.get_loc('soldquantity'+crop+'out')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitssold'+crop+'out')]),'conversionkg']
        data.iloc[i, data.columns.get_loc('sold_outside_kg_'+crop)] = data.iloc[i,data.columns.get_loc('soldquantity'+crop+'out2')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitssold'+crop+'out2')]),'conversionkg']        
        data.iloc[i, data.columns.get_loc('store_kg_'+crop)] = data.iloc[i,data.columns.get_loc('store'+crop+'quantity')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitsstore'+crop)]),'conversionkg']
        data.iloc[i, data.columns.get_loc('lost_kg_'+crop)] = data.iloc[i,data.columns.get_loc('lost'+crop+'quantity')]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitslost'+crop)]),'conversionkg']
        # per plot (wrong)
        #for plot in range(1,N_p+1):
            #data.iloc[i, data.columns.get_loc('kg_'+crop+'_'+plot)] = data.iloc[i,data.columns.get_loc(crop+'perplot_'+plot)]*crop_unit.loc[int(data.iloc[i, data.columns.get_loc('unitsplot'+crop+'_'+plot)]),'conversionkg']
           

for crop in list_crops:
    data['total2_kg_'+crop] = data['sold_kg_'+crop].fillna(0) +data['store_kg_'+crop].fillna(0) +data['lost_kg_'+crop].fillna(0) 
   

#Summary total output kg:
sum_kg = (data[['total_kg_maize', 'total_kg_groundnut', 'total_kg_groundbean', 'total_kg_sweetpotatoe', 'total_kg_fingermillet', 'total_kg_sorghum', 'total_kg_pearlmillet', 'total_kg_soyabean', 'total_kg_pigeonpeas', 'total_kg_cotton', 'total_kg_nkhwani', 'total_kg_cassava',  'total_kg_sugarcane',  'total_kg_tomatoes',  'total_kg_therereokra', 'total_kg_tanaposi']].replace(0,np.nan)).describe(percentiles=percentiles)

print('===============================================================')
print('Check: Distribution of crop production (in kg)')
print('===============================================================')  
sum_kg.dropna(axis=1, how='any')
## STOP RUN


Check: Distribution of crop production (in kg)


,total_kg_maize,total_kg_groundnut,total_kg_groundbean,total_kg_sweetpotatoe,total_kg_fingermillet,total_kg_sorghum,total_kg_pigeonpeas,total_kg_nkhwani,total_kg_tomatoes,total_kg_therereokra
count,106.000000,76.000000,8.000000,10.000000,4.00000,2.000000,44.000000,15.000000,4.000000,11.000000
mean,379.386792,175.065789,27.187500,153.000000,18.12500,51.250000,47.920455,30.166667,18.125000,22.272727
std,597.523740,166.239272,15.892356,217.385985,21.54211,68.942911,53.143503,18.039309,21.347814,23.250122
min,25.000000,10.000000,5.000000,5.000000,2.50000,2.500000,1.000000,5.000000,5.000000,5.000000
5%,81.250000,18.125000,7.625000,14.000000,3.62500,7.375000,5.000000,5.000000,5.375000,5.000000
10%,100.000000,25.000000,10.250000,23.000000,4.75000,12.250000,6.500000,7.000000,5.750000,5.000000
25%,150.000000,50.000000,21.875000,43.750000,8.12500,26.875000,22.500000,16.250000,6.875000,5.000000
50%,250.000000,137.500000,25.000000,100.000000,10.00000,51.250000,35.000000,25.000000,8.750000,12.500000
75%,387.500000,250.000000,31.250000,118.750000,20.00000,75.625000,50.000000,50.000000,20.000000,32.500000
80%,400.000000,300.000000,40.000000,140.000000,26.00000,80.500000,50.000000,50.000000,26.000000,40.000000


In [9]:

# Summary total sellings kg:
sum_sold_kg= (data[['sold_kg_maize', 'sold_kg_groundnut', 'sold_kg_groundbean', 'sold_kg_sweetpotatoe', 'sold_kg_fingermillet', 'sold_kg_sorghum', 'sold_kg_pearlmillet', 'sold_kg_soyabean', 'sold_kg_pigeonpeas', 'sold_kg_cotton', 'sold_kg_nkhwani', 'sold_kg_cassava',  'sold_kg_sugarcane',  'sold_kg_tomatoes',  'sold_kg_therereokra', 'sold_kg_tanaposi']].replace(0,np.nan)).describe(percentiles=percentiles)
print('===============================================================')
print('Check: Distribution of crop Sellings (in kg)')
print('===============================================================')  
sum_sold_kg.dropna(axis=1, how='any')
## STOP RUN

Check: Distribution of crop Sellings (in kg)


,sold_kg_maize,sold_kg_groundnut,sold_kg_sweetpotatoe,sold_kg_pigeonpeas,sold_kg_tomatoes
count,12.000000,28.000000,3.000000,24.000000,2.000000
mean,57.500000,120.535714,54.166667,37.979167,80.000000
std,44.695942,116.470782,43.898557,56.172576,56.568542
min,5.000000,5.000000,12.500000,1.000000,40.000000
5%,5.000000,6.750000,16.250000,4.050000,44.000000
10%,5.500000,10.000000,20.000000,10.000000,48.000000
25%,25.000000,50.000000,31.250000,10.000000,60.000000
50%,50.000000,100.000000,50.000000,20.000000,80.000000
75%,92.500000,112.500000,75.000000,26.250000,100.000000
80%,98.000000,150.000000,80.000000,38.000000,104.000000


In [10]:

#Summary sellings inside kg:
sum_sold_kg_inside = (data[['sold_insiders_kg_maize', 'sold_insiders_kg_groundnut', 'sold_insiders_kg_groundbean', 'sold_insiders_kg_sweetpotatoe', 'sold_insiders_kg_fingermillet', 'sold_insiders_kg_sorghum', 'sold_insiders_kg_pearlmillet', 'sold_insiders_kg_soyabean', 'sold_insiders_kg_pigeonpeas', 'sold_insiders_kg_cotton', 'sold_insiders_kg_nkhwani', 'sold_insiders_kg_cassava',  'sold_insiders_kg_sugarcane',  'sold_insiders_kg_tomatoes',  'sold_insiders_kg_therereokra', 'sold_insiders_kg_tanaposi']].replace(0,np.nan)).describe(percentiles=percentiles)
print('===============================================================')
print('Check: Distribution of crop Sellings to Villagers')
print('===============================================================')  
sum_sold_kg_inside.dropna(axis=1, how='any')
## STOP RUN


Check: Distribution of crop Sellings to Villagers


,sold_insiders_kg_maize,sold_insiders_kg_groundnut,sold_insiders_kg_pigeonpeas,sold_insiders_kg_tomatoes
count,9.000000,6.000000,2.000000,2.000000
mean,54.444444,123.333333,13.000000,70.000000
std,49.965266,133.927841,16.970563,70.710678
min,5.000000,5.000000,1.000000,20.000000
5%,5.000000,5.000000,2.200000,25.000000
10%,5.000000,5.000000,3.400000,30.000000
25%,10.000000,16.250000,7.000000,45.000000
50%,50.000000,75.000000,13.000000,70.000000
75%,90.000000,235.000000,19.000000,95.000000
80%,94.000000,280.000000,20.200000,100.000000


In [11]:

#Summary sellings in village to outsiders
sum_sold_outsiders_kg_in = (data[['sold_outsiders_kg_maize', 'sold_outsiders_kg_groundnut', 'sold_outsiders_kg_groundbean', 'sold_outsiders_kg_sweetpotatoe', 'sold_outsiders_kg_fingermillet', 'sold_outsiders_kg_sorghum', 'sold_outsiders_kg_pearlmillet', 'sold_outsiders_kg_soyabean', 'sold_outsiders_kg_pigeonpeas', 'sold_outsiders_kg_cotton', 'sold_outsiders_kg_nkhwani', 'sold_outsiders_kg_cassava',  'sold_outsiders_kg_sugarcane',  'sold_outsiders_kg_tomatoes',  'sold_outsiders_kg_therereokra', 'sold_outsiders_kg_tanaposi']].replace(0,np.nan)).describe(percentiles=percentiles)
print('===============================================================')
print('Check: Distribution of crop Sellings to Outsiders in Village')
print('===============================================================')  
sum_sold_outsiders_kg_in.dropna(axis=1, how='any')
## STOP RUN


Check: Distribution of crop Sellings to Outsiders in Village


,sold_outsiders_kg_maize,sold_outsiders_kg_groundnut,sold_outsiders_kg_sweetpotatoe,sold_outsiders_kg_pigeonpeas
count,3.000000,20.000000,2.000000,13.000000
mean,66.666667,99.250000,31.250000,41.769231
std,28.867513,92.512801,26.516504,65.114199
min,50.000000,10.000000,12.500000,3.000000
5%,50.000000,10.000000,14.375000,7.200000
10%,50.000000,41.500000,16.250000,10.000000
25%,50.000000,50.000000,21.875000,15.000000
50%,50.000000,100.000000,31.250000,25.000000
75%,75.000000,100.000000,40.625000,30.000000
80%,80.000000,110.000000,42.500000,42.000000


In [12]:

# Summary sellings outside village:
sum_sold_outside_kg_out = (data[['sold_outside_kg_maize', 'sold_outside_kg_groundnut', 'sold_outside_kg_groundbean', 'sold_outside_kg_sweetpotatoe', 'sold_outside_kg_fingermillet', 'sold_outside_kg_sorghum', 'sold_outside_kg_pearlmillet', 'sold_outside_kg_soyabean', 'sold_outside_kg_pigeonpeas', 'sold_outside_kg_cotton', 'sold_outside_kg_nkhwani', 'sold_outside_kg_cassava',  'sold_outside_kg_sugarcane',  'sold_outside_kg_tomatoes',  'sold_outside_kg_therereokra', 'sold_outside_kg_tanaposi']].replace(0,np.nan)).describe(percentiles=percentiles)

# Sum transportation costs
sum_transport_c = (data[['transcostmaizeout', 'transcostgroundnutout', 'transcostgroundbeanout', 'transcostsweetpotatoeout', 'transcostfingermilletout', 'transcostsorghumout', 'transcostpearlmilletout', 'transcostsoyabeanout', 'transcostpigeonpeasout', 'transcostcottonout', 'transcostnkhwaniout', 'transcostcassavaout',  'transcostsugarcaneout',  'transcosttomatoesout',  'transcosttherereokraout', 'transcosttanaposiout']].replace(0,np.nan)).describe(percentiles=percentiles)



# Summary loses kg:
sum_lost_kg= (data[['lost_kg_maize', 'lost_kg_groundnut', 'lost_kg_groundbean', 'lost_kg_sweetpotatoe', 'lost_kg_fingermillet', 'lost_kg_sorghum', 'lost_kg_pearlmillet', 'lost_kg_soyabean', 'lost_kg_pigeonpeas', 'lost_kg_cotton', 'lost_kg_nkhwani', 'lost_kg_cassava',  'lost_kg_sugarcane',  'lost_kg_tomatoes',  'lost_kg_therereokra', 'lost_kg_tanaposi']].replace(0,np.nan)).describe(percentiles=percentiles)
print('===============================================================')
print('Check: Distribution of crop Lost (in kg)')
print('===============================================================')  
sum_lost_kg.dropna(axis=1, how='any')
## STOP RUN

Check: Distribution of crop Lost (in kg)


,lost_kg_maize,lost_kg_groundnut,lost_kg_pigeonpeas,lost_kg_nkhwani,lost_kg_tomatoes
count,17.000000,8.000000,3.000000,6.000000,4.000000
mean,49.470588,35.062500,43.333333,41.666667,13.750000
std,62.590302,68.557816,50.083264,33.565856,10.307764
min,1.000000,1.000000,5.000000,5.000000,5.000000
5%,1.000000,1.350000,7.000000,8.750000,5.000000
10%,1.300000,1.700000,9.000000,12.500000,5.000000
25%,5.000000,2.375000,15.000000,21.250000,5.000000
50%,25.000000,7.500000,25.000000,37.500000,12.500000
75%,50.000000,20.000000,62.500000,50.000000,21.250000
80%,90.000000,34.000000,70.000000,50.000000,22.000000


In [13]:
# Summary Store kg:
sum_store_kg= (data[['store_kg_maize', 'store_kg_groundnut', 'store_kg_groundbean', 'store_kg_sweetpotatoe', 'store_kg_fingermillet', 'store_kg_sorghum', 'store_kg_pearlmillet', 'store_kg_soyabean', 'store_kg_pigeonpeas', 'store_kg_cotton', 'store_kg_nkhwani', 'store_kg_cassava',  'store_kg_sugarcane',  'store_kg_tomatoes',  'store_kg_therereokra', 'store_kg_tanaposi']].replace(0,np.nan)).describe(percentiles=percentiles)
print('===============================================================')
print('Check: Distribution of crop store (in kg)')
print('===============================================================')  
sum_store_kg.dropna(axis=1, how='any')
## STOP RUN


Check: Distribution of crop store (in kg)


,store_kg_maize,store_kg_groundnut,store_kg_groundbean,store_kg_fingermillet,store_kg_pigeonpeas,store_kg_nkhwani,store_kg_therereokra
count,93.000000,64.000000,7.000000,4.000000,40.00000,3.000000,4.000
mean,260.080645,101.796875,108.928571,16.250000,25.23750,6.666667,4.375
std,620.303052,133.053627,227.906492,22.776084,21.76123,2.886751,1.250
min,5.000000,5.000000,5.000000,2.500000,1.00000,5.000000,2.500
5%,25.000000,5.000000,7.250000,2.500000,2.42500,5.000000,2.875
10%,46.000000,6.500000,9.500000,2.500000,4.75000,5.000000,3.250
25%,75.000000,25.000000,18.750000,2.500000,13.75000,5.000000,4.375
50%,125.000000,50.000000,25.000000,6.250000,20.00000,5.000000,5.000
75%,200.000000,125.000000,34.999999,20.000000,25.00000,7.500000,5.000
80%,280.000000,150.000000,40.999999,26.000000,33.00000,8.000000,5.000


In [14]:

# =============================================================================
# Check quantity sold, store, and lost not larger than total
# =============================================================================
for crop in list_crops:
    data['sold_bigger_total_'+crop] = 1*(data['sold_kg_'+crop]> data['total_kg_'+crop]+5)
    data['lost_bigger_total_'+crop] = 1*(data['lost_kg_'+crop]> data['total_kg_'+crop]+5)
    data['store_bigger_total_'+crop] = 1*(data['store_kg_'+crop]> data['total_kg_'+crop]+5)
    data['soldloststore_bigger_total'+crop] = 1*(data['total2_kg_'+crop]>data['total_kg_'+crop]+5)
    
check_sold_bigger_total = data[['sold_bigger_total_maize', 'sold_bigger_total_groundnut', 'sold_bigger_total_groundbean', 'sold_bigger_total_sweetpotatoe', 'sold_bigger_total_fingermillet', 'sold_bigger_total_sorghum', 'sold_bigger_total_pearlmillet', 'sold_bigger_total_soyabean', 'sold_bigger_total_pigeonpeas', 'sold_bigger_total_cotton', 'sold_bigger_total_nkhwani', 'sold_bigger_total_cassava',  'sold_bigger_total_sugarcane',  'sold_bigger_total_tomatoes',  'sold_bigger_total_therereokra', 'sold_bigger_total_tanaposi']]


#Get the households that reported larger amounts than total:

list_hh_check_sell = []
list_hh_check_lost = []
list_hh_check_store = []
list_hh_check = []

for crop in list_crops:
    liers_sell = data.loc[data['sold_bigger_total_'+crop]==1, 'intervieweename']
    liers_store = data.loc[data['store_bigger_total_'+crop]==1, 'intervieweename']
    liers_lost = data.loc[data['lost_bigger_total_'+crop]==1, 'intervieweename']
    liers = data.loc[data['soldloststore_bigger_total_'+crop]==1, 'intervieweename']
    
    list_hh_check_sell.append(liers_sell)
    list_hh_check_store.append(liers_store)
    list_hh_check_lost.append(liers_lost)
    list_hh_check.append(liers)

# sellings check:
hh_to_check_sell = pd.concat(list_hh_check_sell, axis=1)
hh_to_check_sell.columns = list_crops
print('')
print('===============================================================')
print('Check: Households-crop combination where SELLINGS larger than total produced')
print('===============================================================')
hh_to_check_sell.dropna(axis=1)
###STOP RUN


Check: Households-crop combination where SELLINGS larger than total produced


""
20
82
84
86
104


In [15]:
hh_to_check_store = pd.concat(list_hh_check_store, axis=1)
hh_to_check_store.columns = list_crops
print('')
print('===============================================================')
print('Check: Households-crop combination where STORED larger than total produced')
print('===============================================================')
hh_to_check_store.dropna(axis=1).to_string()
### STOP RUN
    


Check: Households-crop combination where STORED larger than total produced


'Empty DataFrame\nColumns: []\nIndex: [5, 70, 78, 82, 104]'

In [16]:

hh_to_check_lost = pd.concat(list_hh_check_lost, axis=1)
hh_to_check_lost.columns = list_crops
print('')
print('===============================================================')
print('Check: Households-crop combination where LOST larger than total produced')
print('===============================================================')
hh_to_check_lost.dropna(axis=1)
### STOP RUN


Check: Households-crop combination where LOST larger than total produced


""
6
56
64
84
103
110


In [17]:

# =============================================================================
#  get PRICES per kg
# =============================================================================
for crop in list_crops:
    data['p_'+crop] = (data['soldvalue'+crop].replace(0,np.nan)).dropna()  / (data['sold_kg_'+crop].replace(0,np.nan)).dropna() 
    #DF = data[['soldvalue'+crop, 'sold_kg_'+crop]].dropna()
sum_prices = data[['p_maize', 'p_groundnut', 'p_groundbean', 'p_sweetpotatoe', 'p_fingermillet', 'p_sorghum', 'p_pearlmillet', 'p_soyabean', 'p_pigeonpeas', 'p_cotton', 'p_nkhwani', 'p_cassava',  'p_sugarcane',  'p_tomatoes',  'p_therereokra', 'p_tanaposi']].describe()
print('') 
print('===============================================================')
print('Check: Distribution of prices')
print('===============================================================')  
sum_prices.dropna(axis=1)


Check: Distribution of prices


,p_maize,p_groundnut,p_sweetpotatoe,p_pigeonpeas,p_tomatoes
count,11.000000,28.000000,3.000000,24.000000,2.000000
mean,130.707071,122.730726,86.666667,297.022421,300.000000
std,41.619300,29.187606,35.118846,301.609591,282.842712
min,77.777778,80.000000,50.000000,20.000000,100.000000
25%,100.000000,100.000000,70.000000,196.200000,200.000000
50%,120.000000,120.000000,90.000000,200.000000,300.000000
75%,150.000000,131.428571,105.000000,242.500000,400.000000
max,200.000000,200.000000,120.000000,1200.000000,500.000000


In [18]:


#Get monetary value:
for crop in list_crops:
    data['y_'+crop] =  np.nanmedian(data['p_'+crop])*data['total_kg_'+crop].fillna(0)
    data['y_agric'] += data['y_'+crop].fillna(0)

#to dollars:

data[['y_agric', 'y_maize', 'y_groundnut', 'y_pigeonpeas']] = 0.0013*data[['y_agric', 'y_maize', 'y_groundnut', 'y_pigeonpeas']]

    
sum_y = data[['y_agric','y_maize', 'y_groundnut', 'y_pigeonpeas']].describe(percentiles=percentiles)
    
## Descriptive statistics
sellings = data[['hhid','soldvaluemaize','soldvaluegroundnut', 'p_maize', 'p_groundnut']]

#%%
# =============================================================================
# Data per plot
# =============================================================================
data = data.stack().apply(pd.to_numeric, errors='ignore').fillna(0).unstack()

# Generate empty dataset
N_p= np.sum(data['total_plots'])
ones = np.ones((int(N_p),2))
data_plots = pd.DataFrame({'hhid':ones[:,0], 'plotid':ones[:,1]})

## Populate dataset with hhid and plotid
i=-1
for hhid in data['hhid']:
    for plot in range(1,int(data.loc[data['hhid']==hhid, 'total_plots'])+1):
        i+=1
        data_plots.iloc[i,0] = hhid
        data_plots.iloc[i,1] = plot
 
       
## generate variables:
# List of chosen crops.. If not chosen then the variables associated to not-chosen crop are unexistent. Update this list
# Everytime we get new data. Check sum_kg for a quick selection.
list_crops_selected = ['maize', 'groundnut', 'sorghum',  'pigeonpeas'] 
# Code also works with all the crops. This is just to avoid empty columns.

for crop in list_crops:
    data_plots[crop+'_kg'] = np.nan
    

data_plots['area'] = np.nan  #area is already converted in acres
data_plots['rentoutplot'] = np.nan
data_plots['valueplot'] = np.nan
data_plots['kg_fertilizer'] = np.nan

#### Loop for plot characteristics
i=-1
for hhid in data['hhid']:
    for plot in range(1,int(data.loc[data['hhid']==hhid, 'total_plots'])+1):
        i+=1
        data_plots.iloc[i, data_plots.columns.get_loc('area')] = float(data.loc[data['hhid']==hhid, 'area_plot_acr_'+str(plot)])
        ## problem: area of rented-in plots. In this case the one with area=0
        data_plots.iloc[i, data_plots.columns.get_loc('rentoutplot')] = float(data.loc[data['hhid']==hhid, 'rentoutplot_'+str(plot)])
        data_plots.iloc[i, data_plots.columns.get_loc('valueplot')] = float(data.loc[data['hhid']==hhid, 'valueplot_'+str(plot)])
        
#### Looop for fertilizer
i=-1
for hhid in data['hhid']:
    for plot in range(1,int(data.loc[data['hhid']==hhid, 'repeatplotsfertilizer_count'])+1):
        i+=1        
        data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,'fertilizerplotsselected_'+str(plot)]))),'kg_fertilizer'] = float(data.loc[data['hhid']==hhid, 'plotkgfertilizer_'+str(plot)])
        
 
#### Loop for crop production       

for hhid in data['hhid']:
        for crop in list_crops:
            for plot in range(1,int(data.loc[data['hhid']==hhid, 'repeatplots'+crop+'_count'])+1):
                #print(data.loc[data['hhid']==hhid,crop+'perplot_'+str(plot)]*crop_unit.loc[int(data.loc[data['hhid']==hhid, 'unitsplot'+crop+'_'+str(plot)]),'conversionkg'])
                data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,crop+'plotsselected_'+str(plot)]))), crop+'_kg'] = float(data.loc[data['hhid']==hhid,crop+'perplot_'+str(plot)]*crop_unit.loc[int(data.loc[data['hhid']==hhid, 'unitsplot'+crop+'_'+str(plot)]),'conversionkg'])
                #data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,crop+'plotsselected_'+str(plot)]))), crop+'_kg'] = i


#### Loop for labor input 
for member in range(1,int(np.max(data['manyhhlaborplot'])+1)):    
    data_plots['months_member_'+str(member)] = np.nan           
    data_plots['weeks_member_'+str(member)] = np.nan             
    data_plots['days_member_'+str(member)] = np.nan 
    data_plots['hours_member_'+str(member)] = np.nan 
    data_plots['hours_member_'+str(member)] = np.nan
for hhid in data['hhid']:
    for member in range(1,int(data.loc[data['hhid']==hhid, 'manyhhlaborplot'])+1):
        for plot in range(1,int(data.loc[data['hhid']==hhid, 'hhlaborperplotrepeat_count_'+str(member)])+1):
                          
            #print(data.loc[data['hhid']==hhid,crop+'perplot_'+str(plot)]*crop_unit.loc[int(data.loc[data['hhid']==hhid, 'unitsplot'+crop+'_'+str(plot)]),'conversionkg'])
                data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,'hhlaborplotsselected_'+str(member)+'_'+str(plot)]))), 'months_member_'+str(member)] = float(data.loc[data['hhid']==hhid,'monthshhplot_'+str(member)+'_'+str(plot)])
                data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,'hhlaborplotsselected_'+str(member)+'_'+str(plot)]))), 'weeks_member_'+str(member)] = float(data.loc[data['hhid']==hhid,'weekshhplot_'+str(member)+'_'+str(plot)])
                data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,'hhlaborplotsselected_'+str(member)+'_'+str(plot)]))), 'days_member_'+str(member)] = float(data.loc[data['hhid']==hhid,'dayshhplot_'+str(member)+'_'+str(plot)])
                data_plots.loc[(data_plots['hhid']==hhid) & (data_plots['plotid']== int(float(data.loc[data['hhid']==hhid,'hhlaborplotsselected_'+str(member)+'_'+str(plot)]))), 'hours_member_'+str(member)] = float(data.loc[data['hhid']==hhid,'hourshhplot_'+str(member)+'_'+str(plot)])

sum_member1 = data_plots['months_member_1', 'weeks_member_1', 'days_member_1', 'hours_member_1'].describe(percentiles=percentiles)
print('===============================================================')
print('reported months,...,hours member 1')
print('===============================================================')
sum_member1
### STOP RUN

TypeError: cannot convert the series to <class 'int'>